In [1]:
import sys

sys.path.append('../src')

In [2]:
def check_cml_env():
    return 'PREPROD'


import os
from typing import List
from dynaconf.base import Settings
from dynaconf import Dynaconf, Validator


ENV = check_cml_env()
ENV_VARS_SECTION = 'env_variables'
YAML_PATH_MAPPING = {
    'UAT': '../config/uat.yaml',
    'PROD': '../config/prod.yaml',
    'PREPROD': '../config/preprod.yaml',
    'NO_CML': '../config/no_cml.yaml'
}

class Config(Dynaconf):

    def __init__(self, 
                 config_yaml_path: str = None, 
                 load_env_vars: bool = True,
                 validators: List[Validator] = None,
                 ):
        if config_yaml_path is None:
            config_yaml_path = YAML_PATH_MAPPING.get(ENV)
        # TODO: Add in validators for certain sections in the config yaml that we think should always be there
        super().__init__(settings_file = config_yaml_path, validators = validators)
        
        self.CONFIG_YAML_PATH = config_yaml_path

        if load_env_vars:
            self.set_env_vars()

    def set_env_vars(self):
        env_variables = getattr(self, ENV_VARS_SECTION, None)
        if env_variables is None:
            warnings.warn(
                f'Skip the setting of environment variables as {ENV_VARS_SECTION} section is not found in {self.CONFIG_YAML_PATH}.'
            )
            return

        for key, value in env_variables.items():
            os.environ[key] = str(value)

    def as_dict(self, env = None, internal = False):
        data = Settings.as_dict(self, env = env, internal = internal)
        data.pop('CONFIG_YAML_PATH', None)
        return data

    to_dict = as_dict
        
    # TODO: Method to write to file each time an attribute value changes/is set
    # TODO: monkeypatch YAML loader in dynaconf loaders so that it uses ruamel yaml which lets us preserve comments and order
    # TODO: Fix the uppercase issue where dynaconf will uppercase all the attributes

In [3]:
config = Config()

In [7]:
from dynaconf import loaders
from dynaconf.loaders import yaml_loader

loaders.write('../config/preprod.yaml', config.as_dict())